In [1]:
import sys
sys.path.append('../..')

import pandas as pd
from pandarallel import pandarallel

from helpers.utils import (
    clean_dataset,
    safe_literal_eval,
    filter_df_by_encode
    )

In [2]:
file_path = '../../output/circum/2024-01-24_SF_circum.parquet'

df = pd.read_parquet(file_path)

In [3]:
intersection_file = '../../helpers/intersection'

with open(intersection_file, "r") as file:
    intersection = file.read().splitlines()

In [4]:
pandarallel.initialize()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
df = clean_dataset(df, intersection)

In [6]:
circum = df[(df.url_flag != "Safe")
                | (df.ref_flag != "Safe")
                | (df.post_flag != "Safe")] 

In [7]:
# Apply the safe conversion to your columns
for col in ['url_leaks', 'referer_leaks', 'post_leaks']:
    circum[col] = circum[col].parallel_apply(safe_literal_eval)

/tmp/ipykernel_293804/429519343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  circum[col] = circum[col].parallel_apply(safe_literal_eval)
/tmp/ipykernel_293804/429519343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  circum[col] = circum[col].parallel_apply(safe_literal_eval)
/tmp/ipykernel_293804/429519343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [8]:
base16_df = filter_df_by_encode(circum, 'base16')
base64_df = filter_df_by_encode(circum, 'base64')
base32_df = filter_df_by_encode(circum, 'base32')
base58_df = filter_df_by_encode(circum, 'base85')
urlencode_df = filter_df_by_encode(circum, 'urlencode')
entity_df = filter_df_by_encode(circum, 'entity')
deflate_df = filter_df_by_encode(circum, 'deflate')
zlib_df = filter_df_by_encode(circum, 'zlib')
gzip_df = filter_df_by_encode(circum, 'gzip')
lzstring = filter_df_by_encode(circum, 'lzstring')
custom_map_1 = filter_df_by_encode(circum, 'custom_map_1')

In [9]:
lzstring[['final_host', 'req_entity', 'url_leaks', 'referer_leaks', 'post_leaks', 'url_flag', 'ref_flag', 'post_flag']].head(10)

,final_host,req_entity,url_leaks,referer_leaks,post_leaks,url_flag,ref_flag,post_flag
56639,shemaroome.com,CleverTap,"[(lzstring, https://www.shemaroome.com/livetv)]",[],None,Circumvention(full),Safe,Safe
89064,madeincookware.com,FullStory,[],"[(madeincookware.com,)]","[(urlencode, collections/sets), (urlencode, ht...",Safe,Safe,Circumvention(full)
271362,aetherapparel.com,"Functional Software, Inc.",[],"[(aetherapparel.com,)]","[(collections/womens-snow-collection,), (urlen...",Safe,Safe,Circumvention(full)
879992,thrillist.com,trx-hub.com,"[(lzstring, https://www.thrillist.com/news)]","[(www.thrillist.com,)]",None,Circumvention(full),Safe,Safe
940623,madeincookware.com,FullStory,[],"[(madeincookware.com,)]","[(urlencode, https://madeincookware.com/collec...",Safe,Safe,Circumvention(full)
1325905,madeincookware.com,FullStory,[],"[(madeincookware.com,)]","[(urlencode, collections/cookware), (urlencode...",Safe,Safe,Circumvention(full)
1572763,fabhotels.com,CleverTap,"[(lzstring, https://www.fabhotels.com/refer-an...","[(www.fabhotels.com,)]",None,Circumvention(full),Safe,Safe
1882729,gardensillustrated.com,trx-hub.com,"[(lzstring, https://www.gardensillustrated.com...","[(www.gardensillustrated.com,)]",None,Circumvention(full),Safe,Safe
1966386,madeincookware.com,FullStory,[],"[(madeincookware.com,)]","[(urlencode, collections/bakeware), (urlencode...",Safe,Safe,Circumvention(full)
2223737,degustapanama.com,CleverTap,"[(lzstring, https://www.degustapanama.com/pana...","[(www.degustapanama.com,)]",None,Circumvention(full),Safe,Safe


In [32]:
lzstring.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

req_entity
CleverTap                    7
trx-hub.com                  3
FullStory                    1
Functional Software, Inc.    1
Storygize                    1
Name: final_host, dtype: int64

In [ ]:
custom_map_1[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,referer_leaks
2809299,mgemi.com,4Cite Marketing,"[(custom_map_1, mgemi.com), (urlencode, https:...","[(mgemi.com,)]"
6690664,mgemi.com,4Cite Marketing,"[(urlencode, pages/men), (custom_map_1, mgemi....","[(mgemi.com,)]"
7705077,mgemi.com,4Cite Marketing,"[(custom_map_1, mgemi.com), (urlencode, https:...","[(mgemi.com,)]"
9916225,mgemi.com,4Cite Marketing,"[(urlencode, pages/rewards), (custom_map_1, mg...","[(mgemi.com,)]"
10134735,mgemi.com,4Cite Marketing,"[(urlencode, collections/leather-accessories),...","[(mgemi.com,)]"


In [31]:
custom_map_1.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

req_entity
4Cite Marketing    1
Name: final_host, dtype: int64

In [ ]:
base16_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,referer_leaks
15365645,homesincolorado.com,datafloat.com,"[(base16, communities)]","[(www.homesincolorado.com,)]"
15365646,homesincolorado.com,datafloat.com,"[(base16, communities)]","[(www.homesincolorado.com,)]"


In [30]:
base16_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

req_entity
datafloat.com    1
Name: final_host, dtype: int64

In [ ]:
base32_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,referer_leaks


In [29]:
base32_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

Series([], Name: final_host, dtype: int64)

In [ ]:
base58_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,referer_leaks


In [28]:
base58_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

Series([], Name: final_host, dtype: int64)

In [ ]:
base64_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,referer_leaks
1558,prabhatkhabar.com,Google LLC,"[(base64, topic/ayodhya-ram-mandir), (base64, ...","[(www.prabhatkhabar.com,)]"
1593,prabhatkhabar.com,Google LLC,"[(base64, topic/ayodhya-ram-mandir), (base64, ...","[(www.prabhatkhabar.com,)]"
1616,prabhatkhabar.com,whistle.mobi,"[(base64, topic/ayodhya-ram-mandir), (urlencod...","[(www.prabhatkhabar.com,)]"
2501,seatunique.com,"Functional Software, Inc.",[],"[(www.seatunique.com,)]"
2838,forkable.com,lfeeder.com,"[(base64, https://forkable.com/terms)]","[(forkable.com,)]"
3906,bizmaker.org,Google LLC,"[(base64, https://www.bizmaker.org/grantsschol...","[(www.bizmaker.org,)]"
3907,bizmaker.org,Google LLC,"[(base64, https://www.bizmaker.org/grantsschol...","[(www.bizmaker.org,)]"
3922,bizmaker.org,Google LLC,"[(base64, https://www.bizmaker.org/grantsschol...","[(www.bizmaker.org,)]"
6107,bemudayana.id,Google LLC,"[(base64, https://bemudayana.id/category/games/)]","[(bemudayana.id,)]"
6123,bemudayana.id,Google LLC,"[(base64, https://bemudayana.id/category/games/)]","[(bemudayana.id,)]"


In [27]:
base64_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

req_entity
Google LLC                        1623
Microsoft Corporation              362
Media.net Advertising FZ-LLC       295
lfeeder.com                         94
RD Gestão e Sistemas Ltda. EPP      73
                                  ... 
unsplash.com                         1
whistle.mobi                         1
wisestamp.com                        1
wsknow.net                           1
xpresslane.in                        1
Name: final_host, Length: 234, dtype: int64

In [ ]:
urlencode_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,referer_leaks
215,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
216,acom.us,Google LLC,"[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
222,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
223,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
224,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
225,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
226,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
227,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
228,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"
229,acom.us,"AdRoll, Inc.","[(acom.us,), (urlencode, https://acom.us/indus...","[(acom.us,)]"


In [26]:
urlencode_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

req_entity
Google LLC               18104
Facebook, Inc.            6572
Microsoft Corporation     3556
Yandex LLC                1221
Twitter, Inc.             1151
                         ...  
0gs25f.ru                    1
00px.net                     1
                             1
北京骐骥云科技有限公司                  1
3anx.net                     1
Name: final_host, Length: 2494, dtype: int64

In [ ]:
entity_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)


,final_host,req_entity,url_leaks,referer_leaks
1641922,electro-master.ru,Yandex LLC,"[(urlencode, section_id=&see_active=Y&see_comp...","[(www.electro-master.ru,)]"
5825054,baginyaforum.ru,Yandex LLC,"[(viewtopic.php,), (baginyaforum.ru,), (urlenc...","[(baginyaforum.ru,)]"
6530346,baginyaforum.ru,Yandex LLC,"[(urlencode, id=123&p=2), (viewtopic.php,), (b...","[(baginyaforum.ru,)]"
7209958,xn--90achb4ac0j.xn--p1ai,Yandex LLC,"[(xn--90achb4ac0j.xn--p1ai,), (urlencode, http...","[(xn--90achb4ac0j.xn--p1ai,)]"
7602552,baginyaforum.ru,Yandex LLC,"[(urlencode, id=104&p=11), (viewtopic.php,), (...","[(baginyaforum.ru,)]"


In [33]:
entity_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

req_entity
Yandex LLC    3
Name: final_host, dtype: int64

In [34]:
deflate_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,post_leaks,referer_leaks


In [35]:
deflate_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

Series([], Name: final_host, dtype: int64)

In [36]:
zlib_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,post_leaks,referer_leaks


In [37]:
zlib_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

Series([], Name: final_host, dtype: int64)

In [38]:
gzip_df[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,post_leaks,referer_leaks


In [39]:
gzip_df.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

Series([], Name: final_host, dtype: int64)

In [40]:
lzstring[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,post_leaks,referer_leaks
56639,shemaroome.com,CleverTap,"[(lzstring, https://www.shemaroome.com/livetv)]",None,[]
89064,madeincookware.com,FullStory,[],"[(urlencode, collections/sets), (urlencode, ht...","[(madeincookware.com,)]"
271362,aetherapparel.com,"Functional Software, Inc.",[],"[(collections/womens-snow-collection,), (urlen...","[(aetherapparel.com,)]"
879992,thrillist.com,trx-hub.com,"[(lzstring, https://www.thrillist.com/news)]",None,"[(www.thrillist.com,)]"
940623,madeincookware.com,FullStory,[],"[(urlencode, https://madeincookware.com/collec...","[(madeincookware.com,)]"
1325905,madeincookware.com,FullStory,[],"[(urlencode, collections/cookware), (urlencode...","[(madeincookware.com,)]"
1572763,fabhotels.com,CleverTap,"[(lzstring, https://www.fabhotels.com/refer-an...",None,"[(www.fabhotels.com,)]"
1882729,gardensillustrated.com,trx-hub.com,"[(lzstring, https://www.gardensillustrated.com...",None,"[(www.gardensillustrated.com,)]"
1966386,madeincookware.com,FullStory,[],"[(urlencode, collections/bakeware), (urlencode...","[(madeincookware.com,)]"
2223737,degustapanama.com,CleverTap,"[(lzstring, https://www.degustapanama.com/pana...",None,"[(www.degustapanama.com,)]"


In [41]:
lzstring.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

req_entity
CleverTap                    7
trx-hub.com                  3
FullStory                    1
Functional Software, Inc.    1
Storygize                    1
Name: final_host, dtype: int64

In [42]:
custom_map_1[['final_host', 'req_entity', 'url_leaks', 'post_leaks' ,'referer_leaks']].head(10)

,final_host,req_entity,url_leaks,post_leaks,referer_leaks
2809299,mgemi.com,4Cite Marketing,"[(custom_map_1, mgemi.com), (urlencode, https:...",None,"[(mgemi.com,)]"
6690664,mgemi.com,4Cite Marketing,"[(urlencode, pages/men), (custom_map_1, mgemi....",None,"[(mgemi.com,)]"
7705077,mgemi.com,4Cite Marketing,"[(custom_map_1, mgemi.com), (urlencode, https:...",None,"[(mgemi.com,)]"
9916225,mgemi.com,4Cite Marketing,"[(urlencode, pages/rewards), (custom_map_1, mg...",None,"[(mgemi.com,)]"
10134735,mgemi.com,4Cite Marketing,"[(urlencode, collections/leather-accessories),...",None,"[(mgemi.com,)]"


In [43]:
custom_map_1.groupby('req_entity').final_host.nunique().sort_values(ascending=False)

req_entity
4Cite Marketing    1
Name: final_host, dtype: int64